In [1]:
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime, ForeignKey, ForeignKeyConstraint
from sqlalchemy.engine.url import URL
from sqlalchemy_utils import database_exists, create_database, drop_database
from sqlalchemy.orm import relationship, declarative_base

user = 'postgres'
password = 'temppass'
host = 'localhost'
port = '5432' 
db_name = 'flights'

server_url = f'postgresql://{user}:{password}@{host}:{port}/'

server_engine = create_engine(server_url)

# Check if the database exists
if database_exists(server_engine.url.set(database=db_name)):
    # Drop the database
    drop_database(server_engine.url.set(database=db_name))
    print(f"Database '{db_name}' has been dropped.")

# Create the database
create_database(server_engine.url.set(database=db_name))
print(f"Database '{db_name}' has been created.")

# Dispose the engine
server_engine.dispose()


Database 'flights' has been dropped.
Database 'flights' has been created.


In [2]:
database_name = 'flights'

engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database_name}')

In [5]:
Base = declarative_base()

class Main_tb(Base):
    __tablename__ = 'main_flights_tb'
    flight_date = Column(DateTime)
    flight_status = Column(String(256))
    dpt_id = Column(String(100), ForeignKey('dim_depart.id'))
    arr_id = Column(String(100), ForeignKey('dim_arrival.id'))
    airline_id = Column(String(5), ForeignKey('airlines.iata'))
    flight_id = Column(String(5), ForeignKey('flights.iata', ondelete='CASCADE'))  # Set ondelete for cascade delete
    aircraft_id = Column(String(5), ForeignKey('aircrafts.iata', ondelete='CASCADE'))  # Set ondelete for cascade delete
    id = Column(String(16), primary_key=True)


class dim_dpt(Base):
    __tablename__ = 'dim_depart'
    airport = Column(String(256))
    timezone = Column(String(32))
    iata = Column(String(3), unique=True)
    icao = Column(String(4), unique=True)
    terminal = Column(String(4))
    gate = Column(String(12))
    delay = Column(Float)
    scheduled = Column(DateTime)
    estimated = Column(DateTime)
    actual = Column(DateTime)
    estimated_runway = Column(DateTime)
    actual_runway = Column(DateTime)
    id = Column(String(100), primary_key=True)

class dim_arr(Base):
    __tablename__ = 'dim_arrival'
    airport = Column(String(256))
    timezone = Column(String(32))
    iata = Column(String(3), unique=True)
    icao = Column(String(4), unique=True)
    terminal = Column(String(4))
    baggage = Column(String(8))
    gate = Column(String(12))
    delay = Column(Float)
    scheduled = Column(DateTime)
    estimated = Column(DateTime)
    actual = Column(DateTime)
    estimated_runway = Column(DateTime)
    actual_runway = Column(DateTime)
    id = Column(String(100), primary_key=True)

class Airline(Base):
    __tablename__ = 'airlines'
    name = Column(String(256))
    iata = Column(String(5), primary_key=True)
    icao = Column(String(5), unique=True)

class flight(Base):
    __tablename__ = 'flights'
    number = Column(Integer)
    iata = Column(String(5), primary_key=True)
    icao = Column(String(5), unique=True)
    codeshared = Column(String(12))

class aircraft(Base):
    __tablename__ = 'aircrafts'
    registration = Column(String(6), unique=True)
    iata = Column(String(5), primary_key=True)
    icao = Column(String(5), unique=True)
    icao24 = Column(String(6), unique=True)   


In [ ]:
Base.metadata.create_all(engine)

In [2]:
import pandas as pd
df = pd.read_csv('data/flights_main.csv')
depart_tb = pd.read_csv('data/dim_dpt.csv')
arrival_tb = pd.read_csv('data/dim_arr.csv')
airline_tb = pd.read_csv('data/dim_airline.csv')


In [6]:

df2 = df.merge(depart_tb, left_on = 'dpt_id', right_on = 'id', how = 'left').merge(arrival_tb, left_on = 'arr_id', right_on = 'id', how = 'left').merge(airline_tb, left_on = 'airline_id', right_on = 'iata', how = 'left')


In [10]:

pd.to_datetime(df2['actual_y']) - pd.to_datetime(df2['actual_x'])

0       0 days 00:59:00
1       0 days 00:59:00
2       0 days 00:59:00
3       0 days 00:59:00
4       0 days 00:59:00
              ...      
39995   0 days 00:32:00
39996   0 days 00:32:00
39997   0 days 00:32:00
39998   0 days 00:32:00
39999   0 days 00:32:00
Length: 40000, dtype: timedelta64[ns]